In [ ]:
import anthropic
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request

client = anthropic.Anthropic()

In [ ]:
def req(id, field):
    return Request(
        custom_id=id,
        params=MessageCreateParamsNonStreaming(
            model="claude-3-7-sonnet-20250219",
            thinking={
                "type": "enabled",
                "budget_tokens": 32000
            },
            max_tokens=64000,
            messages=[{
                "role": "user",
                "content": f"Generate {field} paper that could be published in a top-5 economics journal."
            }]
        )
    )
    
def batch():
    fields = ['a finance', 'a macroeconomics', 'a microeconimics', 'an applied microeconomics']
    reqs =  [ [req(f"{field.split(" ")[1][:3]}_{i}", field) for i in range(5)] for field in fields]
    return [ x for xs in reqs for x in xs ]

In [ ]:
batch = client.messages.batches.create(requests=batch())
print(batch)

In [ ]:
# # send one request
# x = client.messages.batches.create(requests=[
#     req("test", "a finance"),
# ])

In [ ]:
import time

id = ""
def wait(id):
    results = client.messages.batches.retrieve(id).processing_status
    while results == "in_progress":
        stat = client.messages.batches.retrieve(id)
        print(stat.request_counts)
        results = stat.processing_status
        time.sleep(5)

wait(id)

In [ ]:
batches = []
with open("./dump/anthropic-batches.txt", 'r') as f:
    batches = f.read()
    batches = batches.split("\n")

In [ ]:
x = client.messages.batches.results("")
for r in x:
    with open(f'./fake-papers/claude-{r.custom_id}.txt', 'w') as f:
        f.write(r.result.message.content[1].text)
    # print()